In [1]:
import shutil
import os
import base64
from pdf2image import convert_from_path
from PIL import Image
import io
import requests
import json
import pandas as pd

In [2]:
with open('../key.txt', 'r') as file:
    secret_key = file.read().strip()
os.environ['OPENAI_API_KEY'] = secret_key

In [34]:
def convert_pdf_to_images(pdf_path):
    return convert_from_path(pdf_path)[0]

In [4]:
def encode_image_to_base64(image):
    if image.mode == 'RGBA':
        image = image.convert('RGB')
    buffered = io.BytesIO()
    image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

In [12]:
def get_openai_response(base64_image,prompt):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}"
    }
    
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 400
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    if response.status_code == 200:
        response_json = response.json()

        raw_json = response_json['choices'][0]['message']['content']
        response_text = raw_json.replace("json", "").replace("```", "").strip()
        try:
            response_data = json.loads(response_text)
            sbu = response_data.get('sbu')
            invoice_type = response_data.get('invoice_type')
            invoice_no = response_data.get('invoice_no')
            po_number = response_data.get('po_number')
            invoice_date = response_data.get('invoice_date')
            invoice_amount = response_data.get('invoice_amount')
            invoice_tax_amount = response_data.get('invoice_tax_amount')
            delivery_note_number = response_data.get('delivery_note_number')
            # comment = response_data.get('comment')
            currency = response_data.get('currency')
        except json.JSONDecodeError:
            sbu,  delivery_note_number, invoice_no, po_number, invoice_date, invoice_amount, invoice_tax_amount, invoice_type, currency  = None, None, None, None, None, None, None, None, None
        return sbu,  delivery_note_number, invoice_no, po_number, invoice_date, invoice_amount, invoice_tax_amount, invoice_type, currency
    else:
        print(f"Error in OpenAI API response: {response.status_code} - {response.text}")
        return None, None, None, None, None, None, None, None, None

In [23]:
def create_json_output(path_pdf, prompt):
    image_load = convert_pdf_to_images(path_pdf)
    base64_image = encode_image_to_base64(image_load)
    (
        sbu, invoice_type, invoice_no, 
        po_number, invoice_date, invoice_amount, 
        invoice_tax_amount, delivery_note_number, currency
    ) = get_openai_response(base64_image,prompt)

    validated_po_number = []
    for i in po_number:
        if len(i)==10:validated_po_number.append(i)
        else:validated_po_number.append('wrong po')
    
    
    if sbu or invoice_type or invoice_no or po_number  or invoice_date or invoice_amount or invoice_tax_amount or delivery_note_number:
        desired_output = {
            "sbu": sbu,
            "invoice_type": invoice_type,
            "invoice_no": invoice_no,
            "po_number": validated_po_number,
            "invoice_date": invoice_date,
            "invoice_amount": invoice_amount,
            "invoice_tax_amount": invoice_tax_amount,
            "delivery_note_number": delivery_note_number,
            # "comment": comment,
            # "filename": None,
            "currency": currency
        }
        formatted_json = json.dumps(desired_output, indent=4)
        return desired_output

In [27]:
prompt = '''Extract the following details from the invoice:
        sbu
        Invoice Type (Non-Tax Invoices, Tax Invoices, or SVAT Invoices),
        Invoice No,
        PO number,
        Invoice Date,
        Invoice Amount,
        Invoice Tax Amount,
        Delivery note number,
        Currency Type,
        from the image and return the response as a JSON object with
        'sbu'
        'invoice_type',
        'invoice_no',
        'po_number',
        'invoice_date',
        'invoice_amount',
        'invoice_tax_amount',
        'delivery_note_number',
        'currency'
         as keys.

    must return json object only no more any text provide

    1. Invoice Date
        Extract the accurate Invoice Date from the invoice.
    
        Consider alternative names such as:
            Date
            Invoice Date
    
        Invoice Date that could be in various formats such as: 
            10/24/2024, 
            Oct/24/2024, 
            24/10/2024, 
            24/Oct/2024. 
            
        convert it to a uniform format DD/MM/YYYY.
    
    2. Currency Type
        I need to extract the standard currency type used in these invoices. Here's the process to follow:
        1. **Direct Currency Extraction**:
           - Search for common currency symbols (e.g., $, €, £) or currency codes (e.g., USD, EUR, GBP) within the text of the PDF.
           - If a currency is mentioned, extract it as the standard currency type.
        
        2. **Currency Inference from Country**:
           - If no currency is mentioned in the invoice, check for the presence of a country name in the address section.
           - Use the country name to infer the currency type. For example, if the country is "United States," the currency should be USD; if the country is "Germany," the currency should be EUR, etc.
        
        3. **Default Currency**:
           - If neither currency nor country is mentioned in the invoice, default the standard currency type to LKR (Sri Lankan Rupee).

        If the currecy type not mentioned check for the invoice country is mentioned. if mentioned take the 

    3. PO Number        
    
        There could be multiple "PO Numbers".
        
        3.1 In the invoices, "PO Number" can be mentioned using various alternative names:
            "PO number"
            "Purchase order number"
            "Order Number (NO)"
            "Buyer order number"
            "Your order reference number"
            "PO No"
            "Customer PO"
            "Cust. PO No"
            "Manual No"
            "Order Ref"
        
        3.2 Once a "PO Number" is extracted, it should be validated based on the following criteria:
            3a) It must contain exactly 10 characters.
            3b) All characters of the "PO Number" should be digits.
            If the extracted "PO Number" does not meet criteria 3a or 3b, it should be flagged as "wrong PO number".
        
        3.3 Additionally, there are edge cases where the "PO Number" might have more than 10 characters, such as:
            * 2341234562(1748)
            * 2341234562_1748
            * 2341234562 1748   
            
            In these cases, "PO Number" is only upto the special character (i.e., "(", "_", or " "). 
            In this example, the final output should be ["2341234562"]. 
        
        Requirement:
            Identifies all "PO Numbers" based on the alternative names.
            Check for the edge cases.
            Validates each extracted "PO Number" based on the criteria mentioned above.
            Return "PO Number" s as a list
            
    4.Invoice Type
        When considering the Invoice type should follow below details:
    
          4.1 For Tax Invoices:
           
            a. VAT Amount > 0 (≠ null)
            b. VAT % > 0 (≠ null)
            c. Subtotal ≠ Invoice Amount
    
            Consider as a tax invoice if the pair a – b – c is valid.
            
           4.2. For Non-Tax Invoices:
            
            a. VAT amount = 0
            b. VAT % = 0
            c. Subtotal = Invoice Amount
    
            Consider as a non-tax invoice if the pair a – b – c is valid.
    
           4.3. For SVAT Invoices:
            a. SVAT % > 0 (≠ null)
            b. SVAT amount > 0 (≠ null)
           
            Consider as a SVAT invoice if a & b is valid 
    
            alternative names for SVAT are:
                Suspended Vat
                Suspended Tax

    
    5. sbu
        Return None

    
    6. Delivery Note Number Validation
        when considering the 'Delivery note number' should follow below details:
        Extract the accurate Delivery note number from the invoice.
            Consider alternative names such as:
                Delivery note number
                Dispatch note number
                DO Number
                AOD
                Delivery order Number
                DN Number
                Advise No
                Delivery note number ≠ Job No
         
    7. Invoice Amount
    'Invoice Amount' and 'Invoice Tax Amount' retun must be with two decimal places.

    'Invoice Amount' Consider alternative names such as:
        Invoive amount
        Invoice value
        Grand Total	
        Total Amount
        lnvoice Total
        Gross Value

     8. Invoice Number
    'Invoice No' Consider alternative names such as:
        Invoice No
        Invoice number	
        Invoice Ref
        Invoice Reference
        Our reference No
    
    9. Invoice Tax Amount
    'VAT Amount' Consider alternative names such as:
        VAT	Tax	
        Tax amount

    9. Sub Total
    'Sub Total' Consider alternative names such as:
        Net total
        Total	
        Total amount	
        Amount	
        Balance/ amount due	
        Invoice value  
    'Sub Total' cannot Consider as 'Gross Total'


General Rules
If any value is missing, set it to null only.

If any detail is unclear or not sure, please specify an error as an 'comment' in the JSON object. If no error is found, set it to Null.
    '''

In [35]:
# base_folder = "upload\\date inconsistancy"
# base_folder = "upload\\Other currencies"
base_folder = "upload\\prefix suffix"

all_files = [f for f in os.listdir(base_folder) if os.path.isfile(os.path.join(base_folder, f))]
batch_files = [f for f in all_files if f.endswith(('.pdf', '.png', '.jpg', '.jpeg'))]

In [9]:
output_dc = {}
for i in batch_files:
    print(i)
    output_dc[i] = create_json_output(os.path.join(base_folder,i), prompt)

df_abc = pd.DataFrame.from_dict(output_dc, orient='index').reset_index().rename(columns={'index': 'filename'})
df_abc.to_excel(os.path.join(base_folder, "output_fix_v3.xlsx"), index=False)

In [28]:
# base_folder = "upload\\date inconsistancy"
# base_folder = "upload\\Other currencies"
base_folder = "upload\\prefix suffix"

path_pdf = os.path.join(base_folder, '0023 NV FLEXO EXPORT PVT LIMITED 09.08.2023 #PONO 0310020309.pdf')
create_json_output(path_pdf, prompt)

{'sbu': None,
 'invoice_type': 'AD-4795',
 'invoice_no': '2023/24-0023',
 'po_number': ['0310020309'],
 'invoice_date': '30/04/2023',
 'invoice_amount': '546916.43',
 'invoice_tax_amount': '71336.93',
 'delivery_note_number': 'Tax Invoice',
 'currency': 'LKR'}

In [26]:
#